# <font color="#FA8072">Corporación Favorita Grocery Sales Forecasting</font>

## <font color="#FA8072">Importing Libraries<font/>

In [1]:
import os
import pandas as pd
import numpy as np
import gc
import datetime as dt

## <font color="#FA8072">Reading Data</font>

In [4]:
folderPath = "D:\Competitions\CorporationGrocerySalesPrediction"
folderTemp = "D:\Competitions\CorporationGrocerySalesPrediction\temp"

##### Train Data

In [3]:
fileName = "train.csv"
filePath = os.path.join(folderPath,fileName)
types = {
    'item_nbr':np.uint16,
    'store_nbr':np.uint8,
    'unit_sales':np.float32,
    'onpromotion':'object'
}
df_train = pd.read_csv(filePath
                   ,usecols=['store_nbr','date','item_nbr','unit_sales','onpromotion']
                   ,dtype=types
                  )

In [5]:
df_train['onpromotion'] = df_train['onpromotion'].fillna(2)
df_train['onpromotion'] = df_train['onpromotion'].replace('True',1)
df_train['onpromotion'] = df_train['onpromotion'].replace('False',0)
df_train["onpromotion"] = df_train["onpromotion"].astype(np.int8)
df_train["date"] =  pd.to_datetime(df_train["date"],format="%Y-%m-%d",exact=True)

In [6]:
print("Number of rows:",len(df_train))
print("ColumnNames:",df_train.columns.values)

Number of rows: 125497040
ColumnNames: ['date' 'store_nbr' 'item_nbr' 'unit_sales' 'onpromotion']


In [5]:
testFileName = "test.csv"
storeFileName = "stores.csv"
holidayFileName = "holidays_events.csv"
transactionFileName = "transactions.csv"
itemFileName = "items.csv"
oilFileName = "oil.csv"

##### Test data

In [38]:
types = {
    'id':np.uint32,
    'item_nbr':np.uint16,
    'store_nbr':np.uint8,
    'onpromotion':'object'
}
test = pd.read_csv(os.path.join(folderPath,testFileName)
                   ,usecols=['id','store_nbr','date','item_nbr','onpromotion']
                   ,dtype=types)

In [ ]:
test['onpromotion'] = test['onpromotion'].fillna(2)
test['onpromotion'] = test['onpromotion'].replace('True',1)
test['onpromotion'] = test['onpromotion'].replace('False',0)
test["onpromotion"] = test["onpromotion"].astype(np.int8)
test["date"] =  pd.to_datetime(test["date"],format="%Y-%m-%d",exact=True)

#### Store Data

In [6]:
filePath = os.path.join(folderPath,storeFileName)
stores = pd.read_csv(filePath)

In [7]:
stores["store_nbr"] = stores["store_nbr"].astype(np.int8)
stores["type"] = stores["type"].astype("category")
stores["cluster"] = stores["cluster"].astype(np.int8)

In [ ]:
stores.head()

##### Holiday data

In [8]:
filePath = os.path.join(folderPath,holidayFileName)
holidays = pd.read_csv(filePath)
holidays["date"] = pd.to_datetime(holidays.date,format="%Y-%m-%d",exact=True)
holidays["transferred"] = holidays["transferred"].astype(np.bool)

In [9]:
print(holidays.columns)
holidays["locale"].value_counts()

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')


National    174
Local       152
Regional     24
Name: locale, dtype: int64

In [10]:
#picking holidays where day was off
holidays = holidays.loc[(holidays.type!='Event') & (holidays.type!='Work Day') & (holidays.transferred == False)]

In [11]:
holidays.drop(["type","description","transferred"],axis=1,inplace=True)

In [ ]:
holidays.head()

#### Items Data

In [125]:
filePath = os.path.join(folderPath,itemFileName)
items = pd.read_csv(filePath)

In [139]:
print(items.columns)
print(items.family.nunique())
print(items["class"].nunique())

Index(['item_nbr', 'family', 'class', 'perishable'], dtype='object')
33
337


iinfo(min=-32768, max=32767, dtype=int16)

In [140]:
items["item_nbr"] = items["item_nbr"].astype(np.uint16)
items["class"] = items["class"].astype(np.uint16)
items["perishable"] = items["perishable"].astype(np.uint8)
items["family"] = items["family"].astype("category")

In [141]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 4 columns):
item_nbr      4100 non-null uint16
family        4100 non-null category
class         4100 non-null uint16
perishable    4100 non-null uint8
dtypes: category(1), uint16(2), uint8(1)
memory usage: 25.6 KB


### <font color="#FA8072">Data Values</font>

In [22]:
df_train["onpromotion"].value_counts()

0    96028767
2    21657651
1     7810622
Name: onpromotion, dtype: int64

In [9]:
print('Unit Sales smaller than zero:', (df_train["unit_sales"]<0).sum())

Unit Sales smaller than zero: 7795


In [10]:
print("Max date:",df_train["date"].max(),"Min date:",df_train["date"].min())
print("Unique Stores:",len(df_train["store_nbr"].unique()),"Unique Items:",len(df_train["item_nbr"].unique()))

Max date: 2017-08-15 00:00:00 Min date: 2013-01-01 00:00:00
Unique Stores: 54 Unique Items: 3924


In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      uint8
item_nbr       uint16
unit_sales     float32
onpromotion    int8
dtypes: datetime64[ns](1), float32(1), int8(1), uint16(1), uint8(1)
memory usage: 1.9 GB


In [43]:
print(test["onpromotion"].value_counts())
print("Max date:",test["date"].max(),"Min date:",test["date"].min())
print("Unique Stores:",len(test["store_nbr"].unique()),"Unique Items:",len(test["item_nbr"].unique()))

0    3171867
1     198597
Name: onpromotion, dtype: int64
Max date: 2017-08-31 00:00:00 Min date: 2017-08-16 00:00:00
Unique Stores: 54 Unique Items: 256


## <font color="#FA8072">Data Preprocessing</font>

In [143]:
u_dates = df_train.date.unique()
u_stores = df_train.store_nbr.unique()
u_items = df_train.item_nbr.unique()

In [144]:
df_train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
df_train = df_train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)
del u_dates, u_stores, u_items

Exception: cannot handle a non-unique multi-index!

### <font color="#FA8072">Merging Data</font>

##### holidays and store

In [12]:
uniqueStores = stores.store_nbr.unique()
uniqueHolidays = holidays.date.unique()

In [13]:
holiday_store = pd.DataFrame(columns=["store_nbr","holiday"])

In [14]:
for store in uniqueStores:
    for holiday in uniqueHolidays:
        holiday_store = holiday_store.append(pd.DataFrame({'store_nbr':store,'holiday':holiday},index=[0]),ignore_index=True)

In [15]:
del uniqueHolidays,uniqueStores

In [16]:
holiday_store["holiday"] = pd.to_datetime(holiday_store.holiday,format="%Y-%m-%d",exact=True)
holiday_store["store_nbr"] = holiday_store["store_nbr"].astype(np.uint8) 

In [17]:
holiday_store = holiday_store.merge(holidays,how="left",left_on='holiday',right_on="date").drop("date",axis=1)

In [ ]:
holiday_store.head(2)

In [18]:
holiday_store = holiday_store.merge(stores,how="left",left_on='store_nbr',right_on="store_nbr")

In [19]:
del stores,holidays

In [20]:
holiday_store.dayoff = 0

In [21]:
def holiday_estimator(row):
    if row.locale == 'National':
        return 1
    elif row.locale == "Regional":
        if row.state == row.locale_name:
            return 1 
    else:
        if row.city == row.locale_name:
            return 1
        
    return 0

In [22]:
holiday_store["dayoff"] = holiday_store.apply(holiday_estimator,axis=1)

In [23]:
holiday_store.drop(["locale","locale_name",'state','city'],axis=1,inplace=True)
holiday_store["dayoff"] = holiday_store["dayoff"].astype(np.int8)

In [ ]:
holiday_store.to_csv(os.path.join(folderPath,"holiday_store.csv"),index=False)

In [ ]:
holiday_store.head()

## holidays store and training

In [ ]:
df

In [ ]:
df_train = df_train.merge(holiday_store,how="left",left_on=["date","store_nbr"],right_on=["holiday","store_nbr"])

9233